In [2]:
import os
from pathlib import Path
from bifacial_radiance import *
import numpy as np
import pandas as pd

testfolder = Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'AUSTIN'

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
# Create a RadianceObj 'object' named bifacial_example. no whitespace allowed
demo = RadianceObj('tutorial_2', path = str(testfolder))  
albedo = 0.25
demo.setGround(albedo)
# Pull in meteorological data using pyEPW for any global lat/lon
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
# Read in the weather data pulled in above. 
metData = demo.readWeatherFile(weatherFile = epwfile) 

limit_angle = 5 # tracker rotation limit angle. Setting it ridiculously small so this runs faster.
angledelta = 5 # sampling between the limit angles. 
backtrack = True
pitch = 5.0
hub_height = 2.3

x = 1  # meters
y = 2    # meters
moduletype = 'PVmodule'
gcr = 2/pitch

module = demo.makeModule(name=moduletype, x=x, y=y)    

cumulativesky = True # This is important for this example!
trackerdict = demo.set1axis(metdata = demo.readWeatherFile(weatherFile = epwfile) 
, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
trackerdict = demo.genCumSky1axis()



sceneDict = {'pitch':pitch,'hub_height':hub_height, 'nMods': 7, 'nRows': 3}  
trackerdict = demo.makeScene1axis(trackerdict = trackerdict, module = module, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis(trackerdict = trackerdict)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\AUSTIN
path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\AUSTIN
Loading albedo, 1 value(s), 0.250 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos

Module Name: PVmodule
Module PVmodule updated in module.json
Pre-existing .rad file objects\PVmodule.rad will be overwritten

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving fil

In [3]:
trackerdict = demo.analysis1axis(trackerdict, customname = 'Module', sensorsy=9, modWanted=2, rowWanted=1)

trackerdict = demo.calculateResults(agriPV=False)
ResultPVWm2Back = demo.CompiledResults.iloc[0]['Wm2Back']
ResultPVWm2Front = demo.CompiledResults.iloc[0]['Gfront_mean']

# GROUND
resolutionGround = 0.1  # use 1 for faster test runs
numsensors = int((pitch/resolutionGround)+1)

# Analysis for GROUND
modscanback = {'xstart': 0, 
                'zstart': 0.05,
                'xinc': resolutionGround,
                'zinc': 0,
                'Ny':numsensors,
                'orient':'0 0 -1'}

modscanfront = {'Ny':1}    


trackerdict = demo.analysis1axis(trackerdict, customname = 'Ground', modWanted=1, rowWanted=1,
                                    modscanfront=modscanfront, modscanback=modscanback)
trackerdict = demo.calculateResults(agriPV=True)
ResultPVGround = demo.CompiledResults.iloc[0]['Wm2Back']

Linescan in process: 1axis_-5.0Module_Row1_Module2_Front
Linescan in process: 1axis_-5.0Module_Row1_Module2_Back
Saved: results\irr_1axis_-5.0Module_Row1_Module2.csv
Index: -5.0. Wm2Front: 754489.2555555557. Wm2Back: 151522.94444444444
Linescan in process: 1axis_0.0Module_Row1_Module2_Front
Linescan in process: 1axis_0.0Module_Row1_Module2_Back
Saved: results\irr_1axis_0.0Module_Row1_Module2.csv
Index: 0.0. Wm2Front: 1229.933111111111. Wm2Back: 294.7497777777778
Linescan in process: 1axis_5.0Module_Row1_Module2_Front
Linescan in process: 1axis_5.0Module_Row1_Module2_Back
Saved: results\irr_1axis_5.0Module_Row1_Module2.csv
Index: 5.0. Wm2Front: 840604.2111111111. Wm2Back: 158916.4
Bifaciality factor of module stored is  1
Linescan in process: 1axis_-5.0Ground_Row1_Module1_Front
Linescan in process: 1axis_-5.0Ground_Row1_Module1_Back
Saved: results\irr_1axis_-5.0Ground_Row1_Module1_Front.csv
Saved: results\irr_1axis_-5.0Ground_Row1_Module1_Back.csv
Index: -5.0. Wm2Front: 755834.0. Wm2Bac

In [4]:
ResultPVGround

[1270868.445,
 1270839.376,
 1270810.107,
 1270781.037,
 1282425.102,
 1293090.16,
 1290986.3169999998,
 1274016.675,
 1269620.9320000003,
 1289034.5869999998,
 1287190.3660000002,
 1292441.345,
 1293098.724,
 1294423.103,
 1295793.935,
 1306087.4679999999,
 1291227.201,
 1288772.834,
 1301387.368,
 1303541.595,
 1307399.2419999999,
 1315122.3890000002,
 1315920.636,
 1316718.683,
 1297916.324,
 1295070.813,
 1292225.203,
 1289379.6920000003,
 1311262.582,
 1312088.2979999997,
 1313021.237,
 1319224.977,
 1316894.317,
 1314069.3630000001,
 1306720.7989999999,
 1308187.9349999998,
 1314824.8709999998,
 1308856.407,
 1309184.565,
 1309521.477,
 1305348.289,
 1304894.701,
 1301838.113,
 1300586.109,
 1300182.9270000001,
 1299779.7449999999,
 1301989.763,
 1317286.981,
 1314831.3990000002,
 1308527.005,
 1298285.324]

In [ ]:
gid=22
mymonthstart, mymonthend = 1, 12

In [ ]:
xgap = 0

In [ ]:
results = [gid, metData.latitude, metData.longitude, 
           mymonthstart, mymonthend,
           hub_height, pitch, xgap,
           metData.ghi.sum(), 
           ResultPVWm2Front, ResultPVWm2Back, ResultPVGround]

In [ ]:
df_temp = ResultPVGround

In [ ]:
# Tracker Projection of half the module into the ground, for 1-up module in portrait orientation
xp = 10

if pitch == 5.0:
    bedsWanted = 3
else:
    bedsWanted = 6

dist1 = int(np.floor(len(df_temp[xp:-xp])/bedsWanted))

Astart = xp + dist1*0
Bstart = xp + dist1*1
Cstart = xp + dist1*2

if bedsWanted == 6:
    Dstart = xp + dist1*3
    Estart = xp + dist1*4
    Fstart = xp + dist1*5
    Fend = -xp
else:
    Dstart = -xp # in this case it is Cend


testbedA = df_temp[Astart:Bstart]
testbedAmean = np.mean(testbedA)
testbedA_normGHI = testbedAmean / ghi_sum

testbedB = df_temp[Bstart:Cstart]
testbedBmean = np.mean(testbedB)
testbedB_normGHI = testbedBmean / ghi_sum

testbedC = df_temp[Cstart:Dstart]
testbedCmean = np.mean(testbedC)
testbedC_normGHI = testbedCmean / ghi_sum

if bedsWanted == 6:
    testbedD = df_temp[Dstart:Estart]
    testbedDmean = np.mean(testbedD)
    testbedD_normGHI = testbedDmean / ghi_sum

    testbedE = df_temp[Estart:Fstart]
    testbedEmean = np.mean(testbedE)
    testbedE_normGHI = testbedEmean / ghi_sum

    testbedF = df_temp[Fstart:Fend]
    testbedFmean = np.mean(testbedF)
    testbedF_normGHI = testbedFmean / ghi_sum
    
#edgestart = df_temp[:xp]
#edgeend = df_temp[-xp:]

edgemean = np.mean(df_temp[:xp] + df_temp[-xp:])
edge_normGHI = edgemean / ghi_sum

insidemean = np.mean(df_temp[xp:-xp])
inside_normGHI = insidemean / ghi_sum

In [ ]:
# Values to add to returned results:
results = [testbedAmean, testbedBmean, testbedCmean]

In [ ]:
b = [results, results]

In [ ]:
res2 = pd.DataFrame(b, columns=['latitude', 'longitude', 'x'])


In [ ]:
res2

In [ ]:
res = [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
       [462542, 37.810001373291016, -105.86000061035156, 5, 5, 1.5, 5.0, 0.0, 1949797, 2481803.971133333, [190489.65363999997, 179046.10642, 170414.79700000002, 165689.92551, 165288.49835999997, 166262.61995000002, 171109.37050000002, 178816.26449000003, 189029.86281], 
        [549800.4489, 558162.0220000001, 575658.7451000001, 588431.3745000002, 661108.9608, 716042.7550000001, 741429.6590999999, 804642.2954, 837107.5861, 882498.5317000002, 905485.9971999999, 937009.8395000002, 972204.4717999999, 1011749.0571, 1054136.7901, 1069321.791, 1119894.7099, 1151256.7627000003, 1152966.0374999999, 1179310.7300999998, 1199876.3416, 1204969.7530999999, 1230523.3058, 1227668.0707999999, 1226967.2791, 1219370.8270999999, 1214610.4749999999, 1202031.3965999999, 1206640.9608999998, 1185217.3858999999, 1177356.9547000004, 1161197.6721, 1143901.5638, 1110809.0640999998, 1060832.9525, 1015727.9608000001, 980236.9719, 965158.9506000001, 925255.8523, 892081.3624999999, 858106.3715000001, 811867.1314, 757758.3164, 710641.7895000001, 689701.9553, 662002.3923000001, 581206.1259999999, 545761.2799999999, 530173.2996999999, 534446.1595, 539425.5998000001], 663893.3214249999, 1095544.440632258, 1055333.6186900001, 1211787.5795899997, 1026424.1524363637, 0.340493559803918, 0.5618761546110995, 0.5412530733660992, 0.6214942271374916, 0.5264261625371071], [462562, 37.0099983215332, -105.86000061035156, 5, 5, 1.5, 5.0, 0.0, 1945882, 2485181.0611, [189333.61354, 178786.29432, 170578.78020999997, 165947.69356, 164861.04539000004, 166641.35652000003, 171759.08072, 179186.64361000003, 189306.2153], [557252.6761, 577499.9395999999, 592876.5164, 634398.5853000002, 664017.7060999998, 716923.3229, 768075.738, 806721.8362999998, 833998.8709999999, 875040.2607999999, 922543.2886, 958284.4949999999, 997232.9355999999, 1032254.4434999999, 1053950.4624, 1081896.2988999998, 1114498.5705, 1129265.7832999998, 1159335.9251999997, 1197786.7241, 1188640.9851000002, 1197831.9311000002, 1212459.4216, 1218588.8921, 1203523.3844, 1213445.6631, 1195791.8288000003, 1192778.3381999999, 1186067.1162, 1159957.4026, 1149498.5516000001, 1139431.8261000002, 1112218.9976, 1082774.0941, 1044039.8681000001, 1009776.7011000001, 981122.3217, 953290.9429999999, 931453.5220000001, 891146.0404000002, 832626.005, 801290.7955999998, 767585.3720999999, 733128.3038999998, 683797.0108000002, 646730.3396, 609563.0396, 577971.4782999998, 558139.8058999999, 566236.6962000001, 566599.4918000001], 676892.389315, 1088500.4116451612, 1064704.8927099998, 1196908.4963200002, 1011579.8973363638, 0.34785890887268606, 0.5593866491622622, 0.547157994528959, 0.6150981900855242, 0.5198567525350272], [482549, 40.209999084472656, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1695519, 2142260.283588889, [170252.46453, 160478.01144000003, 153690.19762999998, 148559.80352000002, 147413.15671, 147761.2876, 152000.09045, 157945.84322, 165725.54083999997], [504057.539, 521525.82019999996, 513385.6525000001, 543310.1250000001, 587785.2741, 628980.9913000001, 677379.6592999999, 731958.2697000001, 778310.4402, 804712.7474, 807942.6457999999, 838327.5813, 857186.1107000001, 883914.2239999999, 911347.5846000001, 946088.7101, 966669.9276999999, 973287.6989, 991954.8213, 1020393.5439, 1037819.7715, 1040035.7118, 1047639.2277999999, 1040833.6826, 1047477.4548, 1049529.7306000001, 1040444.6764, 1031985.8239, 1015624.4602000001, 988175.5103000001, 980094.6733, 978485.9891000001, 935327.2124999999, 918436.3872, 888968.4438999998, 858067.5701, 818989.6833999999, 785456.0658, 754878.4625, 741082.7113999999, 715664.4608000001, 685147.0908, 670682.1723999999, 623609.2965999999, 599443.4315999998, 556943.6801, 520820.7893, 499180.22650000005, 486927.2277, 493581.44730000006, 500352.92020000005], 596404.74006, 932649.3728451612, 919711.2848299999, 1033956.6049899999, 852313.7872727272, 0.3517534985216916, 0.5500671905447012, 0.5424364367665593, 0.6098171739685606, 0.5026860726849579], [482569, 39.40999984741211, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1715950, 2170561.262555556, [174017.7953, 164767.30560000002, 156907.10890000002, 151603.3428, 150645.8963, 150914.4936, 154503.73669999998, 160359.39179999998, 168192.8297], [522625.9267000001, 529018.7329, 543800.4079, 572267.168, 617850.4211, 659149.0978999999, 702336.5734999999, 734047.6340999999, 780798.4182999999, 813690.3182, 831028.7516, 858550.273, 881716.5869999999, 912449.4792999998, 931326.8851, 947172.0518999998, 965912.8568, 1002770.7137000001, 1014832.1797, 1040464.5236000001, 1043952.7968, 1053628.5172000001, 1048255.6452, 1048108.117, 1046922.1526000001, 1054165.8014999998, 1047757.3677000001, 1041743.1593, 1034441.2895000001, 1017109.2232, 1009668.1564999999, 997621.8571000001, 969315.3498, 935042.3004, 904462.7916999998, 874395.0451999998, 844522.9615, 806574.3539, 774462.969, 753360.7132000001, 726674.7214000002, 694621.9077999999, 675100.0205999999, 632199.9235999997, 604103.9667, 561477.2239000001, 533895.2949, 521675.77280000004, 514996.4101000001, 508855.2298, 528677.4899], 612559.3969350001, 948980.9545612902, 938622.43017, 1043608.407, 872372.8381545455, 0.3569797470409978, 0.5530353183724993, 0.5469987063550803, 0.6081811282380023, 0.5083905930560596], [482589, 38.61000061035156, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1824452, 2330060.7328888886, [180637.92659999998, 169980.5449, 162524.74030000003, 156954.45679999999, 155571.5079, 155971.98949999997, 160014.2746, 166057.40129999997, 175178.24060000002], [540562.5029999999, 527693.8774, 565233.0103999999, 597371.895, 640524.6357999999, 681560.1301, 750282.2281000001, 781636.6871999999, 816793.9282, 848393.4524, 853821.1494000001, 885840.0094000001, 914310.4455999999, 946669.1255999999, 978427.7259999999, 993137.3010999999, 1012939.9119000001, 1042917.1983, 1059019.5835, 1069061.8513999998, 1085369.3261, 1094187.0329999998, 1108218.6279, 1114994.0948, 1102209.4290999998, 1107536.7540999998, 1093386.3964999998, 1097074.9382, 1084762.4423, 1070388.7231, 1053056.7619, 1047605.9831000001, 1012279.3322999999, 989570.2187000001, 946850.6971000001, 931347.7748, 892164.0025, 841083.8404, 800157.275, 784487.5698, 749079.6239, 722289.0307000001, 692955.0376, 651261.7557, 647523.498, 603252.2094, 565545.7514000001, 539846.5598, 536840.5952, 533243.9517000001, 532009.4254], 638741.008125, 992321.1337677417, 975614.4302199999, 1095812.7765099998, 913425.7345000001, 0.3501001989227451, 0.5439009268359715, 0.5347438190864983, 0.6006257092595474, 0.5006575862231509], [482609, 37.810001373291016, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1874080, 2379056.290066667, [185951.23080999998, 175166.58452, 166341.78842999996, 161204.78519, 159630.15701999998, 160787.4192, 164743.6825, 171894.63481999998, 180956.29017], [544021.907, 564848.249, 565467.5134, 600904.2753000001, 611009.4577, 687473.317, 743255.6714, 774862.8354, 836791.7764999999, 868568.2679, 880739.4157000001, 919311.5922000001, 948244.7709999998, 980020.3498, 1011302.6277, 1045479.7415, 1071050.8049, 1100394.3541999997, 1126883.8997999998, 1142514.1389000001, 1136844.8290000004, 1147945.4652000002, 1159563.6002999998, 1159554.4797999999, 1157442.7724, 1169495.6369999999, 1159010.183, 1162753.2557, 1143868.6772, 1130223.8441, 1109266.1657, 1122867.5861, 1080371.2254, 1056431.2622, 998475.4909, 965050.5611999999, 955512.1848999999, 941687.3302999999, 855998.745, 831146.0474, 787643.4961, 762685.7989, 730045.5332, 703089.9707, 673053.9765, 627183.9116999999, 585030.6547, 542009.9971, 527814.2445999999, 550955.3531999999, 546544.6266000001], 652280.86689, 1047003.049503226, 1022594.1695700001, 1152670.27437, 973131.8268363633, 0.348053907458593, 0.5586757499697056, 0.5456512900036286, 0.6150592687451977, 0.5192584237793282], [482629, 37.0099983215332, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1862001, 2378693.0287, [185057.82414, 174508.89815, 166363.17259999996, 160967.63071999996, 159452.74661000003, 160178.89181000003, 164288.85035999998, 170126.55725, 179092.18714999995], [550261.9069000001, 570494.8267, 577492.5259, 617743.3268999999, 661539.2095, 706087.6652, 757182.7927999999, 791346.5379999998, 829964.5688, 864399.0671000002, 897988.3089000001, 937731.7252, 970251.8573, 1006517.0286000001, 1022461.6440999999, 1029228.4742, 1067755.8623, 1080059.3498, 1104028.9712, 1126557.0387000002, 1127263.8721999999, 1134155.6314, 1132596.7906000002, 1146658.4398, 1149691.1349, 1161606.0522, 1138180.5884, 1138488.045, 1126486.0593, 1113185.6985, 1083069.3377999999, 1080469.5868, 1050313.1345, 1010741.4731, 977955.4306999999, 936499.1867000001, 918108.8847000002, 884051.2420999999, 850465.3985, 825473.6458999998, 774901.0845000001, 737592.0391999999, 716399.302, 689731.9114, 656195.3615, 629291.7398, 572331.0146000001, 551582.0094999999, 531854.2826, 526926.5353999999, 537999.3977], 653820.8010750001, 1032352.9347709678, 1024258.02603, 1136831.2312299998, 944731.6732090908, 0.351138802328785, 0.5544319980338184, 0.55008457354749, 0.6105427608416966, 0.5073744177415], [503093, 40.209999084472656, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1786637, 2256472.878777778, [179824.6116, 169914.3319, 161313.7063, 156079.935, 154371.54669999998, 155646.59529999996, 158954.3142, 165310.8694, 174296.54910000003], [507254.27520000003, 509982.36329999997, 518013.66459999996, 538290.5636, 577252.7281999999, 611911.8838999999, 682491.5418, 732703.4779999999, 775560.3934000001, 801688.2298999998, 830186.9375, 855597.2119000001, 889511.9377999998, 916216.1334, 951959.3271999999, 992041.5969, 1007763.0599, 1018144.702, 1059970.8542, 1088372.6913, 1091270.5091, 1115922.1546, 1116871.3616000002, 1127527.8214, 1128727.8895999999, 1124897.4696, 1114018.6042, 1108213.2924, 1119835.0904, 1117061.128, 1100882.77, 1086439.9592999998, 1066287.3871, 1031838.8054999999, 1001980.9535999999, 964573.0773000001, 930648.163, 866377.2080000001, 842565.0512, 824082.1338999999, 798829.5344000001, 744399.6529, 706036.9547, 689599.3027, 664208.2080000001, 610035.2008, 562642.8770999999, 528060.3347, 500426.58640000003, 507822.47130000003, 501182.08560000005], 613478.139805, 1009310.1553645162, 960976.4452100002, 1116434.5320899999, 955864.0948454544, 0.343370331972863, 0.5649217806216462, 0.5378688817090433, 0.624880449744408, 0.5350074440669562], [503113, 39.40999984741211, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1827668, 2319033.0764111113, [182820.22365, 172309.15952000002, 163239.57611999998, 158334.25235, 155679.07641, 156664.13058, 160670.82351000002, 166940.40318, 175512.9604], [523925.48870000005, 522031.7134, 539353.0023999999, 567608.984, 595476.6484000001, 645063.4779, 700209.5074, 748097.1035000001, 811981.9676000001, 845336.8152000001, 861099.0752000001, 892908.7699, 920190.5694999999, 945823.4268, 963832.8633999999, 1001257.0905000002, 1016607.5299000001, 1040887.0354, 1080211.7994, 1101510.7281999998, 1113451.9336, 1113104.3589999997, 1130118.3844, 1129823.4597999998, 1136134.1152000001, 1123862.5272000001, 1125060.6161000002, 1112266.6195, 1105129.0985, 1096467.8041, 1093925.9585, 1079457.5506999998, 1074171.7911, 1016332.9214999999, 981873.2311, 950218.9839999999, 924994.0887999999, 888839.1952999999, 846619.5886, 817112.7969000001, 785259.0806, 720551.6061999999, 694382.9158999999, 684429.6063999999, 659933.0574999999, 618417.7886, 575304.6518999998, 543299.8303, 524063.54509999993, 509224.9307, 523226.42679999996], 627595.953395, 1015114.6126677421, 982432.88882, 1118541.89174, 950800.4715545456, 0.34338619125300657, 0.5554152136316564, 0.5375335612485419, 0.6120049657487028, 0.5202260320553544], [503133, 38.61000061035156, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1920070, 2441784.566466667, [190387.3364, 179615.98148000002, 171211.96129, 164952.49119, 163480.6469, 164204.94385999997, 168016.80827, 175304.61094999997, 184601.66622999997], [542269.4502, 544478.6425000001, 560759.441, 584627.5146999999, 631252.3806, 672339.9489, 724521.5634, 786249.164, 834911.7503000001, 886612.9298999999, 903317.0704, 952089.7240999999, 980526.2825999999, 998440.7585000001, 1031848.9101999999, 1046640.0464999999, 1073655.9616999999, 1095342.9424, 1119139.1790999998, 1138729.9012999998, 1154723.0546, 1167166.3180999998, 1182861.8941, 1177715.8871999998, 1191960.4677, 1195351.7559000002, 1197304.1084, 1185817.7186, 1161419.2646, 1147477.6456, 1135940.1155, 1139089.6985000002, 1114203.5570999999, 1079950.6804, 1041007.8188999998, 1010152.2927999999, 983191.0198, 913100.0146, 879203.1031999998, 856171.4089, 809359.5678000001, 766418.4301, 742908.8363000001, 712908.6542999999, 693263.6118000001, 652669.0939000001, 611205.4473000001, 578238.0663999999, 547895.6247, 545089.9552000001, 540035.0063], 657932.77559, 1066545.1022290322, 1033973.0776799999, 1176179.81148, 996488.1161363636, 0.34266082777711226, 0.5554719891613494, 0.5385080115204133, 0.6125713184831802, 0.5189853058150815], [503153, 37.810001373291016, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1931427, 2461029.1125555555, [190738.9236, 179604.0619, 171771.64919999999, 165720.1788, 163208.21450000003, 163587.90000000002, 167542.6269, 174927.89059999998, 184244.84240000002], [543089.4088000001, 555061.3881000001, 570708.5265000002, 585731.1004, 622341.5763999999, 683258.3473999999, 756182.3936, 790611.4325, 851038.3432, 898893.9772000001, 918621.8707000001, 960775.0885999999, 976217.1471999999, 992978.7976999999, 1024890.6203, 1035042.3256, 1082128.0818, 1119117.4193, 1142130.2975, 1165215.8113, 1176723.6804000002, 1164959.8676, 1199488.1359, 1196618.3495, 1207273.9912999999, 1198194.3361000002, 1193636.5392, 1189716.3399, 1195214.6464, 1170284.3329999999, 1168970.3937000001, 1130941.7209, 1118542.1366, 1074716.6226, 1042338.8488, 1022464.2930000001, 984111.2495, 932011.7871000001, 888433.1859, 853774.7821, 824306.5822, 786962.1299, 756306.3664, 726261.612, 699711.1197000002, 644687.1068, 592061.7714999999, 565032.6566000001, 540049.5877, 545017.1428, 540875.944], 662694.0965750001, 1075801.2671516128, 1041711.7459999999, 1189211.0219300003, 1003691.9638545456, 0.3431111279768793, 0.5569981506687091, 0.5393482363040384, 0.6157162667447438, 0.5196634218401968], [503173, 37.0099983215332, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1938950, 2462256.415844444, [192953.03689999998, 182221.8765, 173387.5619, 167599.9882, 165321.85340000002, 166066.6172, 169884.0125, 176932.266, 186060.01079999996], [548814.47, 565738.9432999999, 579808.1887000002, 603916.4998, 635853.6823999999, 685968.0748, 749301.3654, 801401.1838, 843988.3971999998, 891747.7575000001, 915693.6022000002, 967365.8897, 996046.7468, 1025864.8021999998, 1054882.7393, 1079977.0938000001, 1108996.9106, 1129639.3905999998, 1152021.9661, 1188915.3482, 1193402.0411, 1210726.1511, 1207542.8568000004, 1215947.6695, 1221579.5894000002, 1210430.2592999996, 1225987.7382, 1205756.3447, 1206931.3129999998, 1183751.1983, 1188254.0544, 1159121.7827999997, 1132874.0832, 1100990.3799999997, 1069485.3283000002, 1043791.8425000001, 990573.0426999999, 939488.5466000001, 903450.2051, 868101.5168999999, 814367.9979, 770929.7837000001, 742253.3360000001, 715141.6736000002, 689722.5165000003, 654907.7238, 601520.4327000001, 569576.3197999999, 543241.963, 547947.1054, 544921.3262], 664335.03718, 1093934.1429451613, 1061940.44895, 1208205.51614, 1019136.2527636363, 0.34262618282059876, 0.5641889388303779, 0.5476884132906986, 0.6231236061476573, 0.525612446305287], [523897, 40.209999084472656, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1796744, 2272430.9414333333, [182129.7598, 172189.397, 163599.28089999998, 157177.1025, 155143.55589999998, 155347.2458, 158620.35369999998, 164789.0562, 172782.08779999998], [516548.8956, 527147.7936000001, 533613.1938, 555512.8465999999, 586371.4607, 629716.0412999999, 690651.0119999999, 746778.7919, 794769.2816999999, 848334.9315999999, 866751.0048000001, 904797.6035, 930112.2108, 950098.1793, 970947.1407, 991624.6404999999, 1022659.1592, 1038465.8692999999, 1051731.5853999997, 1074665.3144999999, 1093155.0075, 1113213.4870999998, 1136564.4315000002, 1141089.6215, 1140971.9825000002, 1121297.1266, 1131607.9634, 1109459.1496, 1104651.0547, 1107388.1415, 1092902.8177999998, 1074105.8025, 1034746.6461999998, 1025569.0651999998, 1003636.9412, 944362.9010999999, 907593.4451, 873385.7126, 836962.6698000001, 789457.2390999999, 763374.7782, 728213.6637999999, 701102.9256000001, 672806.4464, 655577.6144000001, 612111.3104, 571754.523, 537636.7653, 522558.1509, 504378.00769999996, 507924.6582], 622175.415725, 1011204.7965387094, 980185.2708, 1119939.79659, 940554.3653454545, 0.34627938967654825, 0.5627984824430801, 0.5455341833895091, 0.6233162857869569, 0.5234771149064389], [523917, 39.40999984741211, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1846905, 2340738.9117666665, [186015.85182, 175481.55875, 166314.54506, 160270.38191, 158101.71744999997, 158287.1832, 161329.9434, 167494.45316000003, 176767.48935], [528004.9975, 544463.6351999999, 543501.3770999999, 559623.3266, 596636.9029999999, 653009.962, 704642.7953, 747507.2266999999, 809705.4327, 849663.4407999999, 873704.3585, 903421.1294, 929293.9857000001, 961904.2008000001, 983419.8704000002, 1025825.2881, 1040211.0497000001, 1072171.3725, 1082920.3869999999, 1102449.8363, 1120093.4962000002, 1135106.3388, 1157509.7348999998, 1164559.0011, 1155988.9723, 1150511.1627, 1147964.9607, 1137343.8831, 1133794.2968000001, 1113511.7973000002, 1118236.2077000001, 1107122.8244, 1085322.2389, 1060691.5481, 1016090.4995, 987785.5296, 943871.1721000001, 898916.5375999999, 869532.9758000001, 818363.1695999999, 784453.6299999999, 749764.4023, 718258.6593999999, 694259.8452, 672932.7034999998, 623607.0654, 577346.7568, 550687.1957999999, 529706.3400000001, 520614.788, 525378.4682], 634965.7660749999, 1034906.1759870967, 997532.1478400001, 1141638.36439, 971853.3030272729, 0.34379990637038715, 0.5603461878045144, 0.5401101560935728, 0.61813594331598, 0.5262064388949474], [523937, 38.61000061035156, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1900430, 2408660.8494888884, [190419.70051000002, 180000.74114, 171396.12163, 164980.64166000002, 162946.71001, 162963.40021000002, 165732.15163000004, 172350.29304, 181069.35263], [547709.2819000001, 550045.1274, 561771.4316000001, 575462.8276, 620062.6193, 662422.9503, 737694.4046, 779947.2229999999, 833848.1399000001, 871254.1017, 898763.5351999998, 943659.2586000001, 966187.5891, 990179.9996, 1013846.1551000001, 1022155.6174, 1060547.0344, 1087837.2904, 1121850.2293999998, 1138078.0646000002, 1150361.2874000003, 1165940.7683, 1175604.1731, 1183598.3243, 1180018.0739999998, 1186820.779, 1175763.7195000001, 1166087.3957000002, 1166902.0596000003, 1159025.0122, 1145874.2031, 1130241.3009, 1113380.5215, 1073966.7829, 1047523.2832999999, 998896.7246999999, 966542.2928, 918951.9064000002, 878187.7226999999, 855149.9809, 798326.1228, 764194.5316999999, 722811.8586, 715879.1723999999, 691289.5498000002, 650477.8503, 605712.3074999999, 569054.2520000001, 549026.7645, 549128.7973000001, 558889.783], 655834.1487199999, 1060653.780932258, 1024310.47738, 1171012.1593099998, 993367.3492727273, 0.345097766673858, 0.5581125223934889, 0.538988795893561, 0.6161827372278904, 0.5227066239076037], [523957, 37.810001373291016, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1916761, 2435751.8013000004, [192729.38594999997, 182013.67349000002, 172858.19317, 166965.45253, 164303.12279999998, 164695.49348, 167982.73245, 174703.68565, 184091.87309000004], [550765.6019, 566119.9637999999, 546663.6394, 569888.6372, 608098.7758, 661245.7606, 720185.9613000001, 784321.6449999999, 838021.7440999999, 896891.733, 926046.9730000002, 969260.7188, 993009.1793999998, 1007332.1109, 1031951.9769000001, 1053376.8939999996, 1090935.7194, 1131054.8456000001, 1132390.9685, 1162960.2826999999, 1159993.0614, 1179798.2410000002, 1181870.1150999998, 1207996.6911, 1207990.117, 1210133.8401, 1203580.3307, 1196060.3535, 1178769.4963, 1164627.6473, 1156459.2139, 1147980.8630000001, 1126101.7221, 1097454.297, 1072763.4755000002, 1045725.8493, 976987.9901999999, 933074.9528, 882229.3730999999, 885516.103, 872113.7133, 777779.4101, 741302.1494999999, 712196.3853999999, 706213.1253000001, 656844.7673999999, 611745.9382999999, 584094.1846000002, 563664.827, 554112.2054, 549234.6349000001], 659969.5544999999, 1083404.7456741936, 1049831.96692, 1189081.98935, 1017855.2321090908, 0.34431499519241043, 0.5652268309268571, 0.5477114605942003, 0.6203600706347844, 0.5310287678584293], [523977, 37.0099983215332, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1963820, 2494998.840444445, [197473.94429999997, 186166.19700000001, 176267.45500000002, 169914.80419999998, 166612.19989999998, 166932.9973, 169765.0231, 176829.1199, 186823.87939999998], [562254.7864, 578773.1328, 593087.5649, 625802.7738, 643814.7714, 695193.6213, 760340.0282999999, 808128.1877999998, 865545.7370999999, 935040.6556999999, 968099.1988, 1010829.8925999999, 1033051.2401000002, 1063984.2307000002, 1081356.5403999998, 1100584.7999, 1121354.6153, 1161167.0627000001, 1173749.881, 1187868.0764, 1189619.2171, 1203474.6373, 1217970.1494, 1223911.2235, 1238730.4556000002, 1243671.9803000002, 1236352.3698, 1218213.4852999998, 1216086.579, 1184159.8719000001, 1166913.1246000002, 1148721.2599000002, 1129297.5911, 1093316.1639, 1071181.1446, 1032044.3538999999, 992862.1183, 937588.0209000001, 891300.8, 866173.4588999999, 821116.885, 756968.5523999999, 729984.7771000002, 692150.7667, 698195.9064999999, 648673.4018000001, 608170.1605, 577870.8724, 570233.2381999999, 559887.2089, 566657.7594], 673838.6951700001, 1104024.2073612902, 1090204.55379, 1217218.99692, 1013683.1746454545, 0.343126506079987, 0.5621819756196037, 0.5551448471804952, 0.6198220798851218, 0.5161792703228679], [544928, 40.209999084472656, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1778594, 2244885.889888889, [182172.34910000002, 172815.64309999996, 163596.88499999998, 157643.66429999997, 155180.9358, 154335.1139, 156677.87960000001, 162831.3097, 170920.0757], [519426.351, 537624.7949999999, 542682.8809999999, 571353.117, 587585.869, 632949.866, 699013.331, 741980.331, 782402.1880000001, 830349.295, 857027.6390000001, 904404.7139999998, 920153.075, 937734.625, 970101.776, 983363.0260000001, 1012081.1409999998, 1039391.6860000001, 1052120.334, 1071168.126, 1082152.8900000001, 1096706.8540000003, 1105962.871, 1117274.442, 1108653.548, 1113834.394, 1094465.3229999999, 1102018.454, 1093909.684, 1086599.54, 1077991.02, 1049558.575, 1025447.9430000001, 1016188.7459999999, 980427.0120000001, 940215.4319000002, 897462.1873999999, 859358.0635000002, 829011.818, 790160.7579999999, 749185.7010000001, 707748.9920000002, 667189.4310000001, 656440.6869999999, 646785.326, 605793.132, 576750.209, 541641.402, 519927.86699999997, 513997.594, 529263.279], 620545.29715, 998842.9483161293, 974754.6142000001, 1100157.8, 928637.0232545454, 0.3488965425217897, 0.5615913178140314, 0.5480478480192782, 0.6185547685418932, 0.5221186078748412], [544948, 39.40999984741211, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1842599, 2339475.5162222227, [187180.1536, 177517.35130000004, 168465.99279999998, 161819.6459, 158947.367, 158376.12050000002, 161086.00430000003, 166684.1408, 174960.569], [550637.395, 568668.496, 586954.058, 597398.7069999999, 615134.046, 671195.235, 722783.027, 760030.007, 843452.784, 882693.3790000001, 908225.7650000001, 944730.912, 948974.886, 981451.5850000001, 1012373.881, 1013398.661, 1042665.442, 1062533.0219999999, 1093362.1209999998, 1093817.591, 1106969.3, 1121778.3930000002, 1132233.525, 1130149.295, 1119268.449, 1116116.134, 1121138.5939999998, 1121660.372, 1113698.322, 1101017.4510000001, 1079268.7940000002, 1078499.68, 1054633.1040000003, 1030865.682, 1006166.511, 968233.7220000001, 912504.1760000001, 879082.5670000002, 845167.4689999999, 821703.8720000001, 767208.6170000001, 711947.8619999998, 685603.0390000001, 686660.9, 671581.953, 620232.485, 592082.265, 566012.562, 540087.237, 551492.677, 560352.0480000001], 649250.0081, 1023512.8353225806, 1010153.3865999999, 1118402.9835, 949394.0176363639, 0.3523555630389466, 0.5554723709947638, 0.5482220421263659, 0.6069703627864772, 0.5152472228826586], [544968, 38.61000061035156, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1881667, 2390350.8116666665, [191069.70739999998, 181042.66369999998, 171662.823, 165044.5997, 161721.92549999995, 161380.1454, 164219.38340000002, 170315.79330000002, 178544.2832], [556826.509, 574782.2930000001, 577841.655, 601879.892, 630773.089, 675178.9799999999, 736039.125, 760331.3049999999, 844312.334, 892058.906, 927749.3069999999, 963477.821, 973990.378, 1004898.368, 1020195.5290000002, 1039349.113, 1054453.704, 1084237.712, 1108777.286, 1124629.7399999998, 1137209.89, 1154465.6439999999, 1151719.637, 1158364.613, 1164836.6500000001, 1160370.243, 1162627.2060000002, 1153538.338, 1151225.975, 1129453.882, 1134196.738, 1112073.519, 1077688.341, 1050452.312, 1019873.7949999999, 1011009.0249999999, 952755.8820000001, 889102.9500000002, 873201.8580000001, 823861.2400000002, 770921.101, 729760.3160000001, 700173.1980000001, 696123.206, 686949.1830000001, 646933.2049999998, 607643.28, 579225.851, 539603.3890000001, 531468.797, 553556.135], 656073.0324, 1049700.2515161291, 1030175.8958, 1152381.2078, 974103.3419090911, 0.34866585447903375, 0.5578565450295557, 0.5474804499414615, 0.6124256883922606, 0.5176810465980916], [544988, 37.810001373291016, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1904114, 2417870.7144444436, [193753.6945, 183601.75350000002, 172637.5467, 165890.39940000002, 163121.0116, 162481.8139, 165004.7277, 170959.1096, 179910.4575], [551995.977, 576789.82, 573419.0539999999, 598033.9989999998, 629552.305, 671679.5129999999, 739384.853, 802524.4299999999, 854174.888, 905682.8630000001, 948071.109, 957855.371, 992988.53, 1021193.9619999999, 1043584.0420000001, 1068806.242, 1073383.22, 1113713.929, 1141732.5240000002, 1137761.8590000002, 1148650.896, 1170827.7949999997, 1175487.9540000001, 1182182.1639999999, 1166124.69, 1190094.089, 1169704.506, 1162573.6130000001, 1153992.9619999998, 1136133.467, 1129653.9179999998, 1108348.5990000002, 1079562.791, 1071163.9039999999, 1030769.2149999999, 985051.9219999999, 957311.0140000001, 907591.422, 858726.6710000001, 852042.0880000001, 774958.9689999999, 725824.236, 708396.8130000001, 687185.4209999999, 679334.0859999999, 633381.523, 576877.498, 563134.4249999999, 543354.808, 555287.4879999999, 546309.562], 656116.1781, 1061614.3044193548, 1049909.0788000003, 1165577.2136, 977743.6830000001, 0.3445782017778347, 0.5575371560838032, 0.5513898216178235, 0.6121362552872359, 0.5134900972315734], [545008, 37.0099983215332, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1946894, 2480780.749555555, [198195.02250000002, 187314.9206, 176407.09950000004, 168885.0414, 165757.8814, 165563.60049999997, 168785.7604, 174708.18240000002, 184361.1931], [565462.406, 584197.605, 593669.927, 619995.783, 656650.0069999999, 703595.0260000001, 770034.3829999999, 820321.0140000002, 853300.443, 921547.8380000001, 948412.4510000001, 991851.389, 1021172.635, 1038063.7509999999, 1067926.564, 1078712.127, 1111205.978, 1158043.0030000003, 1164664.36, 1185703.039, 1195415.8649999998, 1202427.4290000002, 1211714.443, 1213421.0019999999, 1210874.4409999999, 1226035.868, 1199389.987, 1202811.919, 1188188.274, 1161242.6560000002, 1161339.2020000003, 1134860.5209999997, 1124578.8599999999, 1083791.855, 1058264.8490000002, 1038272.3289999999, 1028487.29, 921760.558, 881433.7989999999, 866642.283, 804337.693, 745522.692, 734945.5560000001, 711012.521, 682348.4809999999, 638384.859, 600650.1270000001, 574274.3880000002, 550875.3290000001, 542730.51, 554440.6140000001], 671197.97545, 1092936.9812903227, 1076575.5297, 1201152.1883999999, 1009433.5671818182, 0.3447532199749961, 0.5613746723192545, 0.5529707984615495, 0.6169581848832036, 0.5184840916772142]]


In [ ]:
res2 = [i for i in res if i is not None]

In [ ]:
drop

In [ ]:
type(res)

In [ ]:
res = [ 
       [462542, 37.810001373291016, -105.86000061035156, 5, 5, 1.5, 5.0, 0.0, 1949797, 2481803.971133333, [190489.65363999997, 179046.10642, 170414.79700000002, 165689.92551, 165288.49835999997, 166262.61995000002, 171109.37050000002, 178816.26449000003, 189029.86281], 
        [549800.4489, 558162.0220000001, 575658.7451000001, 588431.3745000002, 661108.9608, 716042.7550000001, 741429.6590999999, 804642.2954, 837107.5861, 882498.5317000002, 905485.9971999999, 937009.8395000002, 972204.4717999999, 1011749.0571, 1054136.7901, 1069321.791, 1119894.7099, 1151256.7627000003, 1152966.0374999999, 1179310.7300999998, 1199876.3416, 1204969.7530999999, 1230523.3058, 1227668.0707999999, 1226967.2791, 1219370.8270999999, 1214610.4749999999, 1202031.3965999999, 1206640.9608999998, 1185217.3858999999, 1177356.9547000004, 1161197.6721, 1143901.5638, 1110809.0640999998, 1060832.9525, 1015727.9608000001, 980236.9719, 965158.9506000001, 925255.8523, 892081.3624999999, 858106.3715000001, 811867.1314, 757758.3164, 710641.7895000001, 689701.9553, 662002.3923000001, 581206.1259999999, 545761.2799999999, 530173.2996999999, 534446.1595, 539425.5998000001], 663893.3214249999, 1095544.440632258, 1055333.6186900001, 1211787.5795899997, 1026424.1524363637, 0.340493559803918, 0.5618761546110995, 0.5412530733660992, 0.6214942271374916, 0.5264261625371071], [462562, 37.0099983215332, -105.86000061035156, 5, 5, 1.5, 5.0, 0.0, 1945882, 2485181.0611, [189333.61354, 178786.29432, 170578.78020999997, 165947.69356, 164861.04539000004, 166641.35652000003, 171759.08072, 179186.64361000003, 189306.2153], [557252.6761, 577499.9395999999, 592876.5164, 634398.5853000002, 664017.7060999998, 716923.3229, 768075.738, 806721.8362999998, 833998.8709999999, 875040.2607999999, 922543.2886, 958284.4949999999, 997232.9355999999, 1032254.4434999999, 1053950.4624, 1081896.2988999998, 1114498.5705, 1129265.7832999998, 1159335.9251999997, 1197786.7241, 1188640.9851000002, 1197831.9311000002, 1212459.4216, 1218588.8921, 1203523.3844, 1213445.6631, 1195791.8288000003, 1192778.3381999999, 1186067.1162, 1159957.4026, 1149498.5516000001, 1139431.8261000002, 1112218.9976, 1082774.0941, 1044039.8681000001, 1009776.7011000001, 981122.3217, 953290.9429999999, 931453.5220000001, 891146.0404000002, 832626.005, 801290.7955999998, 767585.3720999999, 733128.3038999998, 683797.0108000002, 646730.3396, 609563.0396, 577971.4782999998, 558139.8058999999, 566236.6962000001, 566599.4918000001], 676892.389315, 1088500.4116451612, 1064704.8927099998, 1196908.4963200002, 1011579.8973363638, 0.34785890887268606, 0.5593866491622622, 0.547157994528959, 0.6150981900855242, 0.5198567525350272], [482549, 40.209999084472656, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1695519, 2142260.283588889, [170252.46453, 160478.01144000003, 153690.19762999998, 148559.80352000002, 147413.15671, 147761.2876, 152000.09045, 157945.84322, 165725.54083999997], [504057.539, 521525.82019999996, 513385.6525000001, 543310.1250000001, 587785.2741, 628980.9913000001, 677379.6592999999, 731958.2697000001, 778310.4402, 804712.7474, 807942.6457999999, 838327.5813, 857186.1107000001, 883914.2239999999, 911347.5846000001, 946088.7101, 966669.9276999999, 973287.6989, 991954.8213, 1020393.5439, 1037819.7715, 1040035.7118, 1047639.2277999999, 1040833.6826, 1047477.4548, 1049529.7306000001, 1040444.6764, 1031985.8239, 1015624.4602000001, 988175.5103000001, 980094.6733, 978485.9891000001, 935327.2124999999, 918436.3872, 888968.4438999998, 858067.5701, 818989.6833999999, 785456.0658, 754878.4625, 741082.7113999999, 715664.4608000001, 685147.0908, 670682.1723999999, 623609.2965999999, 599443.4315999998, 556943.6801, 520820.7893, 499180.22650000005, 486927.2277, 493581.44730000006, 500352.92020000005], 596404.74006, 932649.3728451612, 919711.2848299999, 1033956.6049899999, 852313.7872727272, 0.3517534985216916, 0.5500671905447012, 0.5424364367665593, 0.6098171739685606, 0.5026860726849579], [482569, 39.40999984741211, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1715950, 2170561.262555556, [174017.7953, 164767.30560000002, 156907.10890000002, 151603.3428, 150645.8963, 150914.4936, 154503.73669999998, 160359.39179999998, 168192.8297], [522625.9267000001, 529018.7329, 543800.4079, 572267.168, 617850.4211, 659149.0978999999, 702336.5734999999, 734047.6340999999, 780798.4182999999, 813690.3182, 831028.7516, 858550.273, 881716.5869999999, 912449.4792999998, 931326.8851, 947172.0518999998, 965912.8568, 1002770.7137000001, 1014832.1797, 1040464.5236000001, 1043952.7968, 1053628.5172000001, 1048255.6452, 1048108.117, 1046922.1526000001, 1054165.8014999998, 1047757.3677000001, 1041743.1593, 1034441.2895000001, 1017109.2232, 1009668.1564999999, 997621.8571000001, 969315.3498, 935042.3004, 904462.7916999998, 874395.0451999998, 844522.9615, 806574.3539, 774462.969, 753360.7132000001, 726674.7214000002, 694621.9077999999, 675100.0205999999, 632199.9235999997, 604103.9667, 561477.2239000001, 533895.2949, 521675.77280000004, 514996.4101000001, 508855.2298, 528677.4899], 612559.3969350001, 948980.9545612902, 938622.43017, 1043608.407, 872372.8381545455, 0.3569797470409978, 0.5530353183724993, 0.5469987063550803, 0.6081811282380023, 0.5083905930560596], [482589, 38.61000061035156, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1824452, 2330060.7328888886, [180637.92659999998, 169980.5449, 162524.74030000003, 156954.45679999999, 155571.5079, 155971.98949999997, 160014.2746, 166057.40129999997, 175178.24060000002], [540562.5029999999, 527693.8774, 565233.0103999999, 597371.895, 640524.6357999999, 681560.1301, 750282.2281000001, 781636.6871999999, 816793.9282, 848393.4524, 853821.1494000001, 885840.0094000001, 914310.4455999999, 946669.1255999999, 978427.7259999999, 993137.3010999999, 1012939.9119000001, 1042917.1983, 1059019.5835, 1069061.8513999998, 1085369.3261, 1094187.0329999998, 1108218.6279, 1114994.0948, 1102209.4290999998, 1107536.7540999998, 1093386.3964999998, 1097074.9382, 1084762.4423, 1070388.7231, 1053056.7619, 1047605.9831000001, 1012279.3322999999, 989570.2187000001, 946850.6971000001, 931347.7748, 892164.0025, 841083.8404, 800157.275, 784487.5698, 749079.6239, 722289.0307000001, 692955.0376, 651261.7557, 647523.498, 603252.2094, 565545.7514000001, 539846.5598, 536840.5952, 533243.9517000001, 532009.4254], 638741.008125, 992321.1337677417, 975614.4302199999, 1095812.7765099998, 913425.7345000001, 0.3501001989227451, 0.5439009268359715, 0.5347438190864983, 0.6006257092595474, 0.5006575862231509], [482609, 37.810001373291016, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1874080, 2379056.290066667, [185951.23080999998, 175166.58452, 166341.78842999996, 161204.78519, 159630.15701999998, 160787.4192, 164743.6825, 171894.63481999998, 180956.29017], [544021.907, 564848.249, 565467.5134, 600904.2753000001, 611009.4577, 687473.317, 743255.6714, 774862.8354, 836791.7764999999, 868568.2679, 880739.4157000001, 919311.5922000001, 948244.7709999998, 980020.3498, 1011302.6277, 1045479.7415, 1071050.8049, 1100394.3541999997, 1126883.8997999998, 1142514.1389000001, 1136844.8290000004, 1147945.4652000002, 1159563.6002999998, 1159554.4797999999, 1157442.7724, 1169495.6369999999, 1159010.183, 1162753.2557, 1143868.6772, 1130223.8441, 1109266.1657, 1122867.5861, 1080371.2254, 1056431.2622, 998475.4909, 965050.5611999999, 955512.1848999999, 941687.3302999999, 855998.745, 831146.0474, 787643.4961, 762685.7989, 730045.5332, 703089.9707, 673053.9765, 627183.9116999999, 585030.6547, 542009.9971, 527814.2445999999, 550955.3531999999, 546544.6266000001], 652280.86689, 1047003.049503226, 1022594.1695700001, 1152670.27437, 973131.8268363633, 0.348053907458593, 0.5586757499697056, 0.5456512900036286, 0.6150592687451977, 0.5192584237793282], [482629, 37.0099983215332, -105.05999755859375, 5, 5, 1.5, 5.0, 0.0, 1862001, 2378693.0287, [185057.82414, 174508.89815, 166363.17259999996, 160967.63071999996, 159452.74661000003, 160178.89181000003, 164288.85035999998, 170126.55725, 179092.18714999995], [550261.9069000001, 570494.8267, 577492.5259, 617743.3268999999, 661539.2095, 706087.6652, 757182.7927999999, 791346.5379999998, 829964.5688, 864399.0671000002, 897988.3089000001, 937731.7252, 970251.8573, 1006517.0286000001, 1022461.6440999999, 1029228.4742, 1067755.8623, 1080059.3498, 1104028.9712, 1126557.0387000002, 1127263.8721999999, 1134155.6314, 1132596.7906000002, 1146658.4398, 1149691.1349, 1161606.0522, 1138180.5884, 1138488.045, 1126486.0593, 1113185.6985, 1083069.3377999999, 1080469.5868, 1050313.1345, 1010741.4731, 977955.4306999999, 936499.1867000001, 918108.8847000002, 884051.2420999999, 850465.3985, 825473.6458999998, 774901.0845000001, 737592.0391999999, 716399.302, 689731.9114, 656195.3615, 629291.7398, 572331.0146000001, 551582.0094999999, 531854.2826, 526926.5353999999, 537999.3977], 653820.8010750001, 1032352.9347709678, 1024258.02603, 1136831.2312299998, 944731.6732090908, 0.351138802328785, 0.5544319980338184, 0.55008457354749, 0.6105427608416966, 0.5073744177415], [503093, 40.209999084472656, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1786637, 2256472.878777778, [179824.6116, 169914.3319, 161313.7063, 156079.935, 154371.54669999998, 155646.59529999996, 158954.3142, 165310.8694, 174296.54910000003], [507254.27520000003, 509982.36329999997, 518013.66459999996, 538290.5636, 577252.7281999999, 611911.8838999999, 682491.5418, 732703.4779999999, 775560.3934000001, 801688.2298999998, 830186.9375, 855597.2119000001, 889511.9377999998, 916216.1334, 951959.3271999999, 992041.5969, 1007763.0599, 1018144.702, 1059970.8542, 1088372.6913, 1091270.5091, 1115922.1546, 1116871.3616000002, 1127527.8214, 1128727.8895999999, 1124897.4696, 1114018.6042, 1108213.2924, 1119835.0904, 1117061.128, 1100882.77, 1086439.9592999998, 1066287.3871, 1031838.8054999999, 1001980.9535999999, 964573.0773000001, 930648.163, 866377.2080000001, 842565.0512, 824082.1338999999, 798829.5344000001, 744399.6529, 706036.9547, 689599.3027, 664208.2080000001, 610035.2008, 562642.8770999999, 528060.3347, 500426.58640000003, 507822.47130000003, 501182.08560000005], 613478.139805, 1009310.1553645162, 960976.4452100002, 1116434.5320899999, 955864.0948454544, 0.343370331972863, 0.5649217806216462, 0.5378688817090433, 0.624880449744408, 0.5350074440669562], [503113, 39.40999984741211, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1827668, 2319033.0764111113, [182820.22365, 172309.15952000002, 163239.57611999998, 158334.25235, 155679.07641, 156664.13058, 160670.82351000002, 166940.40318, 175512.9604], [523925.48870000005, 522031.7134, 539353.0023999999, 567608.984, 595476.6484000001, 645063.4779, 700209.5074, 748097.1035000001, 811981.9676000001, 845336.8152000001, 861099.0752000001, 892908.7699, 920190.5694999999, 945823.4268, 963832.8633999999, 1001257.0905000002, 1016607.5299000001, 1040887.0354, 1080211.7994, 1101510.7281999998, 1113451.9336, 1113104.3589999997, 1130118.3844, 1129823.4597999998, 1136134.1152000001, 1123862.5272000001, 1125060.6161000002, 1112266.6195, 1105129.0985, 1096467.8041, 1093925.9585, 1079457.5506999998, 1074171.7911, 1016332.9214999999, 981873.2311, 950218.9839999999, 924994.0887999999, 888839.1952999999, 846619.5886, 817112.7969000001, 785259.0806, 720551.6061999999, 694382.9158999999, 684429.6063999999, 659933.0574999999, 618417.7886, 575304.6518999998, 543299.8303, 524063.54509999993, 509224.9307, 523226.42679999996], 627595.953395, 1015114.6126677421, 982432.88882, 1118541.89174, 950800.4715545456, 0.34338619125300657, 0.5554152136316564, 0.5375335612485419, 0.6120049657487028, 0.5202260320553544], [503133, 38.61000061035156, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1920070, 2441784.566466667, [190387.3364, 179615.98148000002, 171211.96129, 164952.49119, 163480.6469, 164204.94385999997, 168016.80827, 175304.61094999997, 184601.66622999997], [542269.4502, 544478.6425000001, 560759.441, 584627.5146999999, 631252.3806, 672339.9489, 724521.5634, 786249.164, 834911.7503000001, 886612.9298999999, 903317.0704, 952089.7240999999, 980526.2825999999, 998440.7585000001, 1031848.9101999999, 1046640.0464999999, 1073655.9616999999, 1095342.9424, 1119139.1790999998, 1138729.9012999998, 1154723.0546, 1167166.3180999998, 1182861.8941, 1177715.8871999998, 1191960.4677, 1195351.7559000002, 1197304.1084, 1185817.7186, 1161419.2646, 1147477.6456, 1135940.1155, 1139089.6985000002, 1114203.5570999999, 1079950.6804, 1041007.8188999998, 1010152.2927999999, 983191.0198, 913100.0146, 879203.1031999998, 856171.4089, 809359.5678000001, 766418.4301, 742908.8363000001, 712908.6542999999, 693263.6118000001, 652669.0939000001, 611205.4473000001, 578238.0663999999, 547895.6247, 545089.9552000001, 540035.0063], 657932.77559, 1066545.1022290322, 1033973.0776799999, 1176179.81148, 996488.1161363636, 0.34266082777711226, 0.5554719891613494, 0.5385080115204133, 0.6125713184831802, 0.5189853058150815], [503153, 37.810001373291016, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1931427, 2461029.1125555555, [190738.9236, 179604.0619, 171771.64919999999, 165720.1788, 163208.21450000003, 163587.90000000002, 167542.6269, 174927.89059999998, 184244.84240000002], [543089.4088000001, 555061.3881000001, 570708.5265000002, 585731.1004, 622341.5763999999, 683258.3473999999, 756182.3936, 790611.4325, 851038.3432, 898893.9772000001, 918621.8707000001, 960775.0885999999, 976217.1471999999, 992978.7976999999, 1024890.6203, 1035042.3256, 1082128.0818, 1119117.4193, 1142130.2975, 1165215.8113, 1176723.6804000002, 1164959.8676, 1199488.1359, 1196618.3495, 1207273.9912999999, 1198194.3361000002, 1193636.5392, 1189716.3399, 1195214.6464, 1170284.3329999999, 1168970.3937000001, 1130941.7209, 1118542.1366, 1074716.6226, 1042338.8488, 1022464.2930000001, 984111.2495, 932011.7871000001, 888433.1859, 853774.7821, 824306.5822, 786962.1299, 756306.3664, 726261.612, 699711.1197000002, 644687.1068, 592061.7714999999, 565032.6566000001, 540049.5877, 545017.1428, 540875.944], 662694.0965750001, 1075801.2671516128, 1041711.7459999999, 1189211.0219300003, 1003691.9638545456, 0.3431111279768793, 0.5569981506687091, 0.5393482363040384, 0.6157162667447438, 0.5196634218401968], [503173, 37.0099983215332, -104.26000213623047, 5, 5, 1.5, 5.0, 0.0, 1938950, 2462256.415844444, [192953.03689999998, 182221.8765, 173387.5619, 167599.9882, 165321.85340000002, 166066.6172, 169884.0125, 176932.266, 186060.01079999996], [548814.47, 565738.9432999999, 579808.1887000002, 603916.4998, 635853.6823999999, 685968.0748, 749301.3654, 801401.1838, 843988.3971999998, 891747.7575000001, 915693.6022000002, 967365.8897, 996046.7468, 1025864.8021999998, 1054882.7393, 1079977.0938000001, 1108996.9106, 1129639.3905999998, 1152021.9661, 1188915.3482, 1193402.0411, 1210726.1511, 1207542.8568000004, 1215947.6695, 1221579.5894000002, 1210430.2592999996, 1225987.7382, 1205756.3447, 1206931.3129999998, 1183751.1983, 1188254.0544, 1159121.7827999997, 1132874.0832, 1100990.3799999997, 1069485.3283000002, 1043791.8425000001, 990573.0426999999, 939488.5466000001, 903450.2051, 868101.5168999999, 814367.9979, 770929.7837000001, 742253.3360000001, 715141.6736000002, 689722.5165000003, 654907.7238, 601520.4327000001, 569576.3197999999, 543241.963, 547947.1054, 544921.3262], 664335.03718, 1093934.1429451613, 1061940.44895, 1208205.51614, 1019136.2527636363, 0.34262618282059876, 0.5641889388303779, 0.5476884132906986, 0.6231236061476573, 0.525612446305287], [523897, 40.209999084472656, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1796744, 2272430.9414333333, [182129.7598, 172189.397, 163599.28089999998, 157177.1025, 155143.55589999998, 155347.2458, 158620.35369999998, 164789.0562, 172782.08779999998], [516548.8956, 527147.7936000001, 533613.1938, 555512.8465999999, 586371.4607, 629716.0412999999, 690651.0119999999, 746778.7919, 794769.2816999999, 848334.9315999999, 866751.0048000001, 904797.6035, 930112.2108, 950098.1793, 970947.1407, 991624.6404999999, 1022659.1592, 1038465.8692999999, 1051731.5853999997, 1074665.3144999999, 1093155.0075, 1113213.4870999998, 1136564.4315000002, 1141089.6215, 1140971.9825000002, 1121297.1266, 1131607.9634, 1109459.1496, 1104651.0547, 1107388.1415, 1092902.8177999998, 1074105.8025, 1034746.6461999998, 1025569.0651999998, 1003636.9412, 944362.9010999999, 907593.4451, 873385.7126, 836962.6698000001, 789457.2390999999, 763374.7782, 728213.6637999999, 701102.9256000001, 672806.4464, 655577.6144000001, 612111.3104, 571754.523, 537636.7653, 522558.1509, 504378.00769999996, 507924.6582], 622175.415725, 1011204.7965387094, 980185.2708, 1119939.79659, 940554.3653454545, 0.34627938967654825, 0.5627984824430801, 0.5455341833895091, 0.6233162857869569, 0.5234771149064389], [523917, 39.40999984741211, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1846905, 2340738.9117666665, [186015.85182, 175481.55875, 166314.54506, 160270.38191, 158101.71744999997, 158287.1832, 161329.9434, 167494.45316000003, 176767.48935], [528004.9975, 544463.6351999999, 543501.3770999999, 559623.3266, 596636.9029999999, 653009.962, 704642.7953, 747507.2266999999, 809705.4327, 849663.4407999999, 873704.3585, 903421.1294, 929293.9857000001, 961904.2008000001, 983419.8704000002, 1025825.2881, 1040211.0497000001, 1072171.3725, 1082920.3869999999, 1102449.8363, 1120093.4962000002, 1135106.3388, 1157509.7348999998, 1164559.0011, 1155988.9723, 1150511.1627, 1147964.9607, 1137343.8831, 1133794.2968000001, 1113511.7973000002, 1118236.2077000001, 1107122.8244, 1085322.2389, 1060691.5481, 1016090.4995, 987785.5296, 943871.1721000001, 898916.5375999999, 869532.9758000001, 818363.1695999999, 784453.6299999999, 749764.4023, 718258.6593999999, 694259.8452, 672932.7034999998, 623607.0654, 577346.7568, 550687.1957999999, 529706.3400000001, 520614.788, 525378.4682], 634965.7660749999, 1034906.1759870967, 997532.1478400001, 1141638.36439, 971853.3030272729, 0.34379990637038715, 0.5603461878045144, 0.5401101560935728, 0.61813594331598, 0.5262064388949474], [523937, 38.61000061035156, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1900430, 2408660.8494888884, [190419.70051000002, 180000.74114, 171396.12163, 164980.64166000002, 162946.71001, 162963.40021000002, 165732.15163000004, 172350.29304, 181069.35263], [547709.2819000001, 550045.1274, 561771.4316000001, 575462.8276, 620062.6193, 662422.9503, 737694.4046, 779947.2229999999, 833848.1399000001, 871254.1017, 898763.5351999998, 943659.2586000001, 966187.5891, 990179.9996, 1013846.1551000001, 1022155.6174, 1060547.0344, 1087837.2904, 1121850.2293999998, 1138078.0646000002, 1150361.2874000003, 1165940.7683, 1175604.1731, 1183598.3243, 1180018.0739999998, 1186820.779, 1175763.7195000001, 1166087.3957000002, 1166902.0596000003, 1159025.0122, 1145874.2031, 1130241.3009, 1113380.5215, 1073966.7829, 1047523.2832999999, 998896.7246999999, 966542.2928, 918951.9064000002, 878187.7226999999, 855149.9809, 798326.1228, 764194.5316999999, 722811.8586, 715879.1723999999, 691289.5498000002, 650477.8503, 605712.3074999999, 569054.2520000001, 549026.7645, 549128.7973000001, 558889.783], 655834.1487199999, 1060653.780932258, 1024310.47738, 1171012.1593099998, 993367.3492727273, 0.345097766673858, 0.5581125223934889, 0.538988795893561, 0.6161827372278904, 0.5227066239076037], [523957, 37.810001373291016, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1916761, 2435751.8013000004, [192729.38594999997, 182013.67349000002, 172858.19317, 166965.45253, 164303.12279999998, 164695.49348, 167982.73245, 174703.68565, 184091.87309000004], [550765.6019, 566119.9637999999, 546663.6394, 569888.6372, 608098.7758, 661245.7606, 720185.9613000001, 784321.6449999999, 838021.7440999999, 896891.733, 926046.9730000002, 969260.7188, 993009.1793999998, 1007332.1109, 1031951.9769000001, 1053376.8939999996, 1090935.7194, 1131054.8456000001, 1132390.9685, 1162960.2826999999, 1159993.0614, 1179798.2410000002, 1181870.1150999998, 1207996.6911, 1207990.117, 1210133.8401, 1203580.3307, 1196060.3535, 1178769.4963, 1164627.6473, 1156459.2139, 1147980.8630000001, 1126101.7221, 1097454.297, 1072763.4755000002, 1045725.8493, 976987.9901999999, 933074.9528, 882229.3730999999, 885516.103, 872113.7133, 777779.4101, 741302.1494999999, 712196.3853999999, 706213.1253000001, 656844.7673999999, 611745.9382999999, 584094.1846000002, 563664.827, 554112.2054, 549234.6349000001], 659969.5544999999, 1083404.7456741936, 1049831.96692, 1189081.98935, 1017855.2321090908, 0.34431499519241043, 0.5652268309268571, 0.5477114605942003, 0.6203600706347844, 0.5310287678584293], [523977, 37.0099983215332, -103.45999908447266, 5, 5, 1.5, 5.0, 0.0, 1963820, 2494998.840444445, [197473.94429999997, 186166.19700000001, 176267.45500000002, 169914.80419999998, 166612.19989999998, 166932.9973, 169765.0231, 176829.1199, 186823.87939999998], [562254.7864, 578773.1328, 593087.5649, 625802.7738, 643814.7714, 695193.6213, 760340.0282999999, 808128.1877999998, 865545.7370999999, 935040.6556999999, 968099.1988, 1010829.8925999999, 1033051.2401000002, 1063984.2307000002, 1081356.5403999998, 1100584.7999, 1121354.6153, 1161167.0627000001, 1173749.881, 1187868.0764, 1189619.2171, 1203474.6373, 1217970.1494, 1223911.2235, 1238730.4556000002, 1243671.9803000002, 1236352.3698, 1218213.4852999998, 1216086.579, 1184159.8719000001, 1166913.1246000002, 1148721.2599000002, 1129297.5911, 1093316.1639, 1071181.1446, 1032044.3538999999, 992862.1183, 937588.0209000001, 891300.8, 866173.4588999999, 821116.885, 756968.5523999999, 729984.7771000002, 692150.7667, 698195.9064999999, 648673.4018000001, 608170.1605, 577870.8724, 570233.2381999999, 559887.2089, 566657.7594], 673838.6951700001, 1104024.2073612902, 1090204.55379, 1217218.99692, 1013683.1746454545, 0.343126506079987, 0.5621819756196037, 0.5551448471804952, 0.6198220798851218, 0.5161792703228679], [544928, 40.209999084472656, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1778594, 2244885.889888889, [182172.34910000002, 172815.64309999996, 163596.88499999998, 157643.66429999997, 155180.9358, 154335.1139, 156677.87960000001, 162831.3097, 170920.0757], [519426.351, 537624.7949999999, 542682.8809999999, 571353.117, 587585.869, 632949.866, 699013.331, 741980.331, 782402.1880000001, 830349.295, 857027.6390000001, 904404.7139999998, 920153.075, 937734.625, 970101.776, 983363.0260000001, 1012081.1409999998, 1039391.6860000001, 1052120.334, 1071168.126, 1082152.8900000001, 1096706.8540000003, 1105962.871, 1117274.442, 1108653.548, 1113834.394, 1094465.3229999999, 1102018.454, 1093909.684, 1086599.54, 1077991.02, 1049558.575, 1025447.9430000001, 1016188.7459999999, 980427.0120000001, 940215.4319000002, 897462.1873999999, 859358.0635000002, 829011.818, 790160.7579999999, 749185.7010000001, 707748.9920000002, 667189.4310000001, 656440.6869999999, 646785.326, 605793.132, 576750.209, 541641.402, 519927.86699999997, 513997.594, 529263.279], 620545.29715, 998842.9483161293, 974754.6142000001, 1100157.8, 928637.0232545454, 0.3488965425217897, 0.5615913178140314, 0.5480478480192782, 0.6185547685418932, 0.5221186078748412], [544948, 39.40999984741211, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1842599, 2339475.5162222227, [187180.1536, 177517.35130000004, 168465.99279999998, 161819.6459, 158947.367, 158376.12050000002, 161086.00430000003, 166684.1408, 174960.569], [550637.395, 568668.496, 586954.058, 597398.7069999999, 615134.046, 671195.235, 722783.027, 760030.007, 843452.784, 882693.3790000001, 908225.7650000001, 944730.912, 948974.886, 981451.5850000001, 1012373.881, 1013398.661, 1042665.442, 1062533.0219999999, 1093362.1209999998, 1093817.591, 1106969.3, 1121778.3930000002, 1132233.525, 1130149.295, 1119268.449, 1116116.134, 1121138.5939999998, 1121660.372, 1113698.322, 1101017.4510000001, 1079268.7940000002, 1078499.68, 1054633.1040000003, 1030865.682, 1006166.511, 968233.7220000001, 912504.1760000001, 879082.5670000002, 845167.4689999999, 821703.8720000001, 767208.6170000001, 711947.8619999998, 685603.0390000001, 686660.9, 671581.953, 620232.485, 592082.265, 566012.562, 540087.237, 551492.677, 560352.0480000001], 649250.0081, 1023512.8353225806, 1010153.3865999999, 1118402.9835, 949394.0176363639, 0.3523555630389466, 0.5554723709947638, 0.5482220421263659, 0.6069703627864772, 0.5152472228826586], [544968, 38.61000061035156, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1881667, 2390350.8116666665, [191069.70739999998, 181042.66369999998, 171662.823, 165044.5997, 161721.92549999995, 161380.1454, 164219.38340000002, 170315.79330000002, 178544.2832], [556826.509, 574782.2930000001, 577841.655, 601879.892, 630773.089, 675178.9799999999, 736039.125, 760331.3049999999, 844312.334, 892058.906, 927749.3069999999, 963477.821, 973990.378, 1004898.368, 1020195.5290000002, 1039349.113, 1054453.704, 1084237.712, 1108777.286, 1124629.7399999998, 1137209.89, 1154465.6439999999, 1151719.637, 1158364.613, 1164836.6500000001, 1160370.243, 1162627.2060000002, 1153538.338, 1151225.975, 1129453.882, 1134196.738, 1112073.519, 1077688.341, 1050452.312, 1019873.7949999999, 1011009.0249999999, 952755.8820000001, 889102.9500000002, 873201.8580000001, 823861.2400000002, 770921.101, 729760.3160000001, 700173.1980000001, 696123.206, 686949.1830000001, 646933.2049999998, 607643.28, 579225.851, 539603.3890000001, 531468.797, 553556.135], 656073.0324, 1049700.2515161291, 1030175.8958, 1152381.2078, 974103.3419090911, 0.34866585447903375, 0.5578565450295557, 0.5474804499414615, 0.6124256883922606, 0.5176810465980916], [544988, 37.810001373291016, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1904114, 2417870.7144444436, [193753.6945, 183601.75350000002, 172637.5467, 165890.39940000002, 163121.0116, 162481.8139, 165004.7277, 170959.1096, 179910.4575], [551995.977, 576789.82, 573419.0539999999, 598033.9989999998, 629552.305, 671679.5129999999, 739384.853, 802524.4299999999, 854174.888, 905682.8630000001, 948071.109, 957855.371, 992988.53, 1021193.9619999999, 1043584.0420000001, 1068806.242, 1073383.22, 1113713.929, 1141732.5240000002, 1137761.8590000002, 1148650.896, 1170827.7949999997, 1175487.9540000001, 1182182.1639999999, 1166124.69, 1190094.089, 1169704.506, 1162573.6130000001, 1153992.9619999998, 1136133.467, 1129653.9179999998, 1108348.5990000002, 1079562.791, 1071163.9039999999, 1030769.2149999999, 985051.9219999999, 957311.0140000001, 907591.422, 858726.6710000001, 852042.0880000001, 774958.9689999999, 725824.236, 708396.8130000001, 687185.4209999999, 679334.0859999999, 633381.523, 576877.498, 563134.4249999999, 543354.808, 555287.4879999999, 546309.562], 656116.1781, 1061614.3044193548, 1049909.0788000003, 1165577.2136, 977743.6830000001, 0.3445782017778347, 0.5575371560838032, 0.5513898216178235, 0.6121362552872359, 0.5134900972315734], [545008, 37.0099983215332, -102.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1946894, 2480780.749555555, [198195.02250000002, 187314.9206, 176407.09950000004, 168885.0414, 165757.8814, 165563.60049999997, 168785.7604, 174708.18240000002, 184361.1931], [565462.406, 584197.605, 593669.927, 619995.783, 656650.0069999999, 703595.0260000001, 770034.3829999999, 820321.0140000002, 853300.443, 921547.8380000001, 948412.4510000001, 991851.389, 1021172.635, 1038063.7509999999, 1067926.564, 1078712.127, 1111205.978, 1158043.0030000003, 1164664.36, 1185703.039, 1195415.8649999998, 1202427.4290000002, 1211714.443, 1213421.0019999999, 1210874.4409999999, 1226035.868, 1199389.987, 1202811.919, 1188188.274, 1161242.6560000002, 1161339.2020000003, 1134860.5209999997, 1124578.8599999999, 1083791.855, 1058264.8490000002, 1038272.3289999999, 1028487.29, 921760.558, 881433.7989999999, 866642.283, 804337.693, 745522.692, 734945.5560000001, 711012.521, 682348.4809999999, 638384.859, 600650.1270000001, 574274.3880000002, 550875.3290000001, 542730.51, 554440.6140000001], 671197.97545, 1092936.9812903227, 1076575.5297, 1201152.1883999999, 1009433.5671818182, 0.3447532199749961, 0.5613746723192545, 0.5529707984615495, 0.6169581848832036, 0.5184840916772142]]


In [ ]:
type(res)

In [1]:
import pandas as pd
import pickle

In [2]:
dffile = r'C:\Users\sayala\Documents\GitHub\Studies\res_gencum1axis\Pitch_5_Colorado.pkl'

In [4]:
asdf = r'C:\Users\sayala\Documents\GitHub\Studies\RUN.pkl'

In [6]:
foo = pd.read_pickle(asdf)
len(foo)

1077

In [9]:
pd.__version__

'1.4.4'

In [10]:
from platform import python_version
print(python_version())

3.9.13


In [7]:
df = pd.read_pickle(dffile)
df

,gid,latitude,longitude,MonthStart,MonthEnd,hub_height,pitch,xgap,ghi_sum,ResultPVWm2Front,...,edgemean,insidemean,testbedAmean,testbedBmean,testbedCmean,edge_normGHI,inside_normGHI,testbedA_normGHI,testbedB_normGHI,testbedC_normGHI
0,389877,38.450001,-109.059998,5,5,1.5,5.0,0.0,1827671,2.342394e+06,...,649014.98005,1.016027e+06,9.410084e+05,1.103454e+06,1.004746e+06,0.355105,0.555913,0.514867,0.603749,0.549741
1,389881,38.290001,-109.059998,5,5,1.5,5.0,0.0,1880502,2.408586e+06,...,659420.40250,1.053802e+06,9.730318e+05,1.146895e+06,1.042601e+06,0.350662,0.560383,0.517432,0.609887,0.554427
2,393321,40.849998,-108.900002,5,5,1.5,5.0,0.0,1760906,2.258396e+06,...,613019.20079,9.956603e+05,9.076415e+05,1.087392e+06,9.922851e+05,0.348127,0.565425,0.515440,0.617518,0.563508
3,393325,40.689999,-108.900002,5,5,1.5,5.0,0.0,1792327,2.275458e+06,...,628579.22475,1.009642e+06,9.260745e+05,1.098530e+06,1.004804e+06,0.350706,0.563313,0.516688,0.612907,0.560614
4,393329,40.529999,-108.900002,5,5,1.5,5.0,0.0,1790827,2.286670e+06,...,629414.22520,1.016506e+06,9.233647e+05,1.108731e+06,1.017340e+06,0.351466,0.567618,0.515608,0.619117,0.568084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,557665,37.650002,-102.180000,5,5,1.5,5.0,0.0,1914359,2.435057e+06,...,663095.24355,1.072901e+06,1.056367e+06,1.177191e+06,9.931218e+05,0.346380,0.560449,0.551812,0.614927,0.518775
1073,557669,37.490002,-102.180000,5,5,1.5,5.0,0.0,1915466,2.427051e+06,...,662650.72230,1.079297e+06,1.058254e+06,1.190080e+06,9.977163e+05,0.345948,0.563465,0.552479,0.621300,0.520874
1074,557673,37.330002,-102.180000,5,5,1.5,5.0,0.0,1928551,2.461948e+06,...,670615.22040,1.082065e+06,1.084141e+06,1.179458e+06,9.916393e+05,0.347730,0.561077,0.562153,0.611577,0.514189
1075,557677,37.169998,-102.180000,5,5,1.5,5.0,0.0,1919237,2.436401e+06,...,661001.22360,1.075550e+06,1.064267e+06,1.185310e+06,9.860238e+05,0.344408,0.560405,0.554526,0.617595,0.513758


In [9]:
respi = r'C:\Users\sayala\Documents\GitHub\Studies\389881_hh1.5_rtr5.0_xgap0.0_from_5TO5\results.pkl'


[389881,
 38.290000915527344,
 -109.05999755859375,
 5,
 5,
 1.5,
 5.0,
 0.0,
 1880502,
 2416626.113555555,
 [176997.16689999998,
  167689.1122,
  162916.6025,
  161029.27039999998,
  162650.56550000003,
  166263.3494,
  173416.8767,
  182220.80510000003,
  191611.24860000002],
 [557432.8939999999,
  544970.01,
  573228.691,
  581249.9969999999,
  610456.936,
  642973.5249999999,
  689194.249,
  704542.5469999999,
  721239.0380000001,
  747155.101,
  799900.3790000001,
  856315.127,
  885301.0990000002,
  899611.624,
  968030.3429999999,
  1009817.8970000002,
  1023483.162,
  1080635.747,
  1083671.34,
  1097002.231,
  1107956.3990000002,
  1132726.3490000002,
  1135854.5779999997,
  1156038.464,
  1153404.594,
  1180103.48,
  1151431.9699999997,
  1168072.5899999999,
  1154785.637,
  1139152.487,
  1137583.02,
  1111844.294,
  1087843.748,
  1076737.2230000002,
  1050736.631,
  1033119.2520000001,
  1023644.684,
  1009557.102,
  997512.1079999999,
  970670.0680000001,
  944353.568,
  

In [11]:
uno

,gid,latitude,longitude,MonthStart,MonthEnd,hub_height,pitch,xgap,ghi_sum,ResultPVWm2Front,...,edgemean,insidemean,testbedAmean,testbedBmean,testbedCmean,edge_normGHI,inside_normGHI,testbedA_normGHI,testbedB_normGHI,testbedC_normGHI
0,389881,38.290001,-109.059998,5,5,1.5,5.0,0.0,1880502,2.416626e+06,...,658835.65745,1.052481e+06,970376.8949,1.147953e+06,1.040327e+06,0.350351,0.559681,0.516020,0.610450,0.553218
1,393321,40.849998,-108.900002,5,5,1.5,5.0,0.0,1760906,2.254001e+06,...,608984.35405,9.985862e+05,911619.7273,1.091857e+06,9.928549e+05,0.345836,0.567087,0.517699,0.620054,0.563832


In [8]:
len(foo)

1077

In [3]:
df = pd.read_pickle(dffile)

In [4]:
df

,gid,latitude,longitude,MonthStart,MonthEnd,hub_height,pitch,xgap,ghi_sum,ResultPVWm2Front,...,edgemean,insidemean,testbedAmean,testbedBmean,testbedCmean,edge_normGHI,inside_normGHI,testbedA_normGHI,testbedB_normGHI,testbedC_normGHI
0,389881,38.290001,-109.059998,5,5,1.5,5.0,0.0,1880502,2.416626e+06,...,658835.65745,1.052481e+06,970376.8949,1.147953e+06,1.040327e+06,0.350351,0.559681,0.516020,0.610450,0.553218
1,393321,40.849998,-108.900002,5,5,1.5,5.0,0.0,1760906,2.254001e+06,...,608984.35405,9.985862e+05,911619.7273,1.091857e+06,9.928549e+05,0.345836,0.567087,0.517699,0.620054,0.563832


In [ ]:
407351, 407371, 407391, 407411, 407431, 425231, 425251, 425271,
425291, 425311, 443525, 443545, 443565, 443585, 443605, 462482,
462502, 462522, 462542, 462562, 482549, 482569, 482589, 482609,
482629, 503093, 503113, 503133, 503153, 503173, 523897, 523917,
523937, 523957, 523977, 544928, 544948, 544968, 544988, 545008

In [ ]:
df.keys()

In [ ]:
res2 = pd.DataFrame(res2, columns=[
        'gid', 'latitude', 'longitude', 'MonthStart', 'MonthEnd',
        'hub_height', 'pitch', 'xgap', 'ghi_sum',
        'ResultPVWm2Front', 'ResultPVWm2Back', 'ResultPVGround',
        'edgemean', 'insidemean', 'testbedAmean', 'testbedBmean', 
        'testbedCmean', 'edge_normGHI', 'inside_normGHI',
        'testbedA_normGHI', 'testbedB_normGHI', 'testbedC_normGHI', 
        ])

In [5]:
res  = [
[394281, 37.290000915527344, -108.86000061035156, 5, 5, 1.5, 5.0, 0.0, 1939152, 2474774.4376666667, 
[180245.16920000003, 170990.22549999997, 165369.1443, 163513.746, 165395.3909, 169579.04340000002, 176848.44150000002, 185881.41019999998, 196401.9209], 
[577748.275, 561196.136, 571398.0809999999, 580208.3339999998, 610593.958, 646178.1799999999, 691668.816, 693050.4890000001, 706443.1799999999, 754635.607, 797385.3080000001, 853092.694, 876772.643, 888211.7549999999, 989409.124, 1021838.6639999999, 1065430.484, 1103807.8979999998, 1131458.8579999998, 1145600.4719999998, 1163886.7840000002, 1155262.4849999999, 1180262.5179999997, 1170491.0330000003, 1194535.544, 1189307.3949999998, 1205785.557, 1195744.9080000003, 1200270.2179999999, 1181321.915, 1174637.769, 1163163.625, 1163884.726, 1129442.0669999998, 1115517.613, 1088059.243, 1070753.9819999998, 1054195.425, 1052950.89, 1021336.201, 943080.1669999999, 910452.4909999999, 859636.9669999997, 833430.0799999998, 740017.1969999999, 669640.245, 632840.2509999999, 608197.238, 595433.2860000001, 568949.637, 556575.1240000001], 668414.6786, 1086674.1279032258, 987300.7899999998, 1183686.8357000002, 1088820.1552727271, 0.3446943192694539, 0.5603862553854602, 0.5091404851192686, 0.6104146738883801, 0.561492938806616], 
[398640, 37.290000915527344, -108.66000366210938, 5, 5, 1.5, 5.0, 0.0, 1943979, 2490914.194666667, 
[181647.2945, 172125.4164, 166594.5221, 164592.21929999997, 165583.52349999995, 169695.2783, 176902.0491, 185982.37820000004, 196694.52719999998], 
[549160.144, 529409.09, 561199.118, 582167.9460000001, 612806.1409999998, 651185.1390000002, 688010.199, 703085.947, 725736.332, 725674.4479999999, 810930.432, 858947.9219999999, 883331.351, 945525.984, 983770.16, 1017985.884, 1053174.3299999998, 1102353.628, 1123393.2829999998, 1140930.405, 1161187.751, 1171155.4239999999, 1193643.048, 1201573.2399999998, 1207136.134, 1211132.644, 1197312.622, 1193752.625, 1189963.79, 1180222.396, 1160493.98, 1143214.3499999999, 1162064.941, 1126607.234, 1117301.876, 1087625.566, 1078880.267, 1072322.09, 1039261.912, 1000495.7769999999, 953029.93, 906732.964, 876446.596, 834598.784, 744691.6059999999, 703165.32, 629676.571, 602797.801, 599535.9789999999, 557909.632, 553831.613], 666891.0684999998, 1089313.5798709677, 992034.3378999999, 1190707.9674, 1085572.5384545454, 0.34305466700000353, 0.560352544894244, 0.5103112419938692, 0.6125107150848852, 0.5584281200849112]
]

In [18]:
from datetime import datetime
now = datetime.now()


In [13]:
from datetime import datetime

In [15]:
now

datetime.datetime(2023, 3, 23, 10, 9, 27, 217806)

In [6]:
type(res)

list

In [7]:
np.array(res)

C:\Users\sayala\AppData\Local\Temp\1\ipykernel_20580\4118355402.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(res)


array([[394281, 37.290000915527344, -108.86000061035156, 5, 5, 1.5, 5.0,
        0.0, 1939152, 2474774.4376666667,
        list([180245.16920000003, 170990.22549999997, 165369.1443, 163513.746, 165395.3909, 169579.04340000002, 176848.44150000002, 185881.41019999998, 196401.9209]),
        list([577748.275, 561196.136, 571398.0809999999, 580208.3339999998, 610593.958, 646178.1799999999, 691668.816, 693050.4890000001, 706443.1799999999, 754635.607, 797385.3080000001, 853092.694, 876772.643, 888211.7549999999, 989409.124, 1021838.6639999999, 1065430.484, 1103807.8979999998, 1131458.8579999998, 1145600.4719999998, 1163886.7840000002, 1155262.4849999999, 1180262.5179999997, 1170491.0330000003, 1194535.544, 1189307.3949999998, 1205785.557, 1195744.9080000003, 1200270.2179999999, 1181321.915, 1174637.769, 1163163.625, 1163884.726, 1129442.0669999998, 1115517.613, 1088059.243, 1070753.9819999998, 1054195.425, 1052950.89, 1021336.201, 943080.1669999999, 910452.4909999999, 859636.9669999997, 833

In [ ]:
import pandas as pd

In [ ]:
res2 = pd.DataFrame(res, columns=[
        'gid', 'latitude', 'longitude', 'MonthStart', 'MonthEnd',
        'hub_height', 'pitch', 'xgap', 'ghi_sum',
        'ResultPVWm2Front', 'ResultPVWm2Back', 'ResultPVGround',
        'edgemean', 'insidemean', 'testbedAmean', 'testbedBmean', 
        'testbedCmean', 'edge_normGHI', 'inside_normGHI',
        'testbedA_normGHI', 'testbedB_normGHI', 'testbedC_normGHI', 
        ])

In [ ]:
res2